In [1]:
import os

import pandas as pd
import time
import pickle

In [2]:

from ase.calculators.espresso import Espresso, EspressoProfile
from ase.io.espresso import read_espresso_out

In [3]:
from disalloy.cellfunc import jitter_ia, jitter_ic, compute_distortion
from disalloy.grid_run import run_packet

In [24]:
profile = EspressoProfile(
    command='/qe/q-e/bin/pw.x', pseudo_dir='/qe/q-e/pseudo/SSSP/'
)

pseudopotentials = {
    'Al':'Al.pbe-n-kjpaw_psl.1.0.0.UPF',
    'Co':'Co_pbe_v1.2.uspp.F.UPF',
    'Cr':'cr_pbe_v1.5.uspp.F.UPF',
    'Cu':'Cu.paw.z_11.ld1.psl.v1.0.0-low.upf',
    'Fe':'Fe.pbe-spn-kjpaw_psl.0.2.1.UPF',
    'Mn':'mn_pbe_v1.5.uspp.F.UPF',
    'Ni':'ni_pbe_v1.4.uspp.F.UPF',
    'V':'v_pbe_v1.4.uspp.F.UPF',
     }


input_data = {
    'system': {'ecutwfc': 60, 'ecutrho': 480},
    'disk_io': 'low',  # Automatically put into the 'control' section
    'tstress':True,  # deprecated, put in input_data
    'tprnfor':True,  # deprecated, put in input_data
    'calculation': 'vc-relax',#'scf',
    'conv_thr':1e-8,
    'occupations':'smearing',
    'smearing':'mp',
    'degauss':0.02,
    'mixing_beta':0.7
}

calc_espresso_relax = Espresso(
    profile=profile,
    pseudopotentials=pseudopotentials,
    input_data=input_data,
    kspacing=0.5e-1,
)

def relaxer_espresso(row):
    structure = row['init_structure'].copy()
    structure.calc = calc_espresso_relax
    e = structure.get_potential_energy()
    atoms_clean = structure.copy()
    atoms_clean.calculator = None
    return atoms_clean, e
def relaxer_espresso_jitter_ia(row, ia = 0.01):
    structure = row['init_structure'].copy()
    structure = jitter_ia(structure, ia)
    structure.calc = calc_espresso_relax
    e = structure.get_potential_energy()
    atoms_clean = structure.copy()
    atoms_clean.calculator = None
    return atoms_clean, e
def relaxer_espresso_jitter_ic(row, ic = 0.01):
    structure = row['init_structure'].copy()
    structure = jitter_ic(structure, ic)
    structure.calc = calc_espresso_relax
    e = structure.get_potential_energy()
    atoms_clean = structure.copy()
    atoms_clean.calculator = None
    return atoms_clean, e

In [16]:
from disalloy.grid_run import run_packet

In [6]:
with open('AlCoCrCuFeMnNiV_5_10.pkl', 'rb') as f:
    structure_DB = pickle.load(f)

In [17]:
df = structure_DB.copy()
# if os.path.exists('df_calc.pkl'):
#     df = pd.read_pickle('df_calc.pkl')

In [18]:
relaxer_dic_espresso = {'calculator_label': 'relaxer_espresso',
                            'relaxer': relaxer_espresso}

In [19]:
print(df.attrs)

{'number_of_structures': {1: 8, 2: 56, 3: 336, 4: 4046, 5: 15904, 6: 17080, 7: 23968, 8: 128540, 9: 140000, 10: 180000}, 'is_exhaustive': {1: True, 2: True, 3: True, 4: True, 5: True, 6: False, 7: False, 8: False, 9: False, 10: False}}


In [20]:
ti = time.time()
run_packet('df_espresso', df, relaxer_dic_espresso, 10, 1)
print('{}s'.format(time.time() - ti))

2024-09-04 02:15:25 - starting packet run: attempting 10 runs of size 1

2024-09-04 02:19:10 - index of ran strctures:
 [0, 1, 2, 3, 4, 5, 6, 7]

2024-09-04 02:19:10 - Succesfuly ran packet of 8 runs in 184.9036304950714s

224.81355381011963s


In [ ]:
ti = time.time()
run_packet('df_espresso', df, relaxer_dic_espresso, 10, 2)
print('{}s'.format(time.time() - ti))

2024-09-04 02:19:29 - starting packet run: attempting 10 runs of size 2



In [23]:
df[df[relaxer_dic_espresso['calculator_label']].notnull()]

,init_structure,size,relaxer_espresso
0,"(Atom('Al', [0.0, 0.0, 0.0], index=0))",1,"((Atom('Al', [0.0, 0.0, 0.0], index=0)), -537...."
1,"(Atom('Co', [0.0, 0.0, 0.0], index=0))",1,"((Atom('Co', [0.0, 0.0, 0.0], index=0)), -4060..."
2,"(Atom('Cr', [0.0, 0.0, 0.0], index=0))",1,"((Atom('Cr', [0.0, 0.0, 0.0], index=0)), -2387..."
3,"(Atom('Cu', [0.0, 0.0, 0.0], index=0))",1,"((Atom('Cu', [0.0, 0.0, 0.0], index=0)), -2899..."
4,"(Atom('Fe', [0.0, 0.0, 0.0], index=0))",1,"((Atom('Fe', [0.0, 0.0, 0.0], index=0)), -4479..."
5,"(Atom('Mn', [0.0, 0.0, 0.0], index=0))",1,"((Atom('Mn', [0.0, 0.0, 0.0], index=0)), -2873..."
6,"(Atom('Ni', [0.0, 0.0, 0.0], index=0))",1,"((Atom('Ni', [0.0, 0.0, 0.0], index=0)), -4670..."
7,"(Atom('V', [0.0, 0.0, 0.0], index=0))",1,"((Atom('V', [0.0, 0.0, 0.0], index=0)), -1973...."
9,"(Atom('Al', [0.0, 0.0, 0.0], index=0), Atom('C...",2,"((Atom('Al', [0.0, 0.0, 0.0], index=0), Atom('..."
12,"(Atom('Al', [0.0, 0.0, 0.0], index=0), Atom('M...",2,"((Atom('Al', [0.0, 0.0, 0.0], index=0), Atom('..."


In [25]:
relaxer_dic_espresso_jitter_ic = {
    'calculator_label': 'relaxer_espresso_jitter_ic',
    'relaxer': relaxer_espresso_jitter_ic}

In [ ]:
ti = time.time()
run_packet('df_espresso', df, relaxer_dic_espresso_jitter_ic, 10, 1,
          copy_calculated=relaxer_dic_espresso['calculator_label'])
print('{}s'.format(time.time() - ti))

2024-09-04 02:52:13 - starting packet run: attempting 10 runs of size 1



In [ ]:
ti = time.time()
run_packet('df_espresso', df, relaxer_dic_espresso_jitter_ic, 10, 2,
          copy_calculated=relaxer_dic_espresso['calculator_label'])
print('{}s'.format(time.time() - ti))

In [ ]:
calculated_index = df[df[relaxer_dic_espresso['calculator_label']].notnull()].index

In [ ]:
for i, j, k  in zip(df.loc[calculated_index, 'init_structure'], 
                df.loc[calculated_index, 'relaxer_espresso'].array, 
             df.loc[calculated_index, 'relaxer_espresso_jitter_ic'].array):
    print(compute_distortion(i, j[0]))
    print(compute_distortion(i, k[0]))
    print()